In [1]:
# %load_ext autoreload
# %autoreload 2

# Version Notes: 

### v1: 
* add Data_HourMinute for all exported datasources


# Tip for quick search

* Needs attention: the place where needs update or better logic
* question to be answered: the place where things are still not clear
* Manual Check: Unit test where you can drill in to find the data that leads to the check results for a specific project and specific check
* TODO: things needs to be done
* bookmark: stop point from last visit


# Admin Notes:


1. The AMTool dataset is archived daily as csv files and used for the project book check. 
The csv files are located at: 
r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Tableau Dashboards\DataLake'

2. The excel input files are checked daily and archived with datestamp whenever it is modified.
The continuously updated excel input files are located at: r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\excel'
The excel input file are archived at: r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Tableau Dashboards\Data_MiscInput'
To recover the archived excel file used in project book check for a target date, select the excel file with latest datestamp but is still earlier than the target date.

3. The check summary export action is logged daily. It can be used for daily monitoring. 
The file export log is located at: \\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\output_internal\log

4. The published data are at:

    * csv files for district asset manager: http://svgcshopp.dot.ca.gov/DataLake/ProjectBookCheck/
    * csv files for HQ AM: \\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\output_internal
    * tableau workbook with live data source: https://tableau.dot.ca.gov/#/site/AssetManagement/workbooks/1815/views


# General Approach

use Minor raw data as basis for data checks. 
Each project only occupies one line

can expand columns, only if it will not create duplicate rows in the SHOPP raw dataset. 


# Data clean process

* funding amount: remove dollar sign, 
* fill missing value, string, numerical, 
* remove leading single quote for string value
* strip off leading and trailing space 

* regulate column names




# Import common modules

<a id='TableOfContents'></a>

# Table Of Contents

## Data Preprocessing

### [Global Constants](#GlobalConstants)


### [Load and cleanup source data](#Read_Data)


## Add fields to SHOPP raw data (calculate and join)
* [Calculated Fields](#AddDataColumns)
* [Join Tables](#DataJoining)



## Data Check and Export


## [Data Check List](#Issue_Table1)
The main table of check issues, 
one issue per row, 


* [Will_this_project_be_included_in_the_Project_Book](#Will_this_project_be_included_in_the_Project_Book)
* [Does_project_cost_exceed_Minor_Program_limits](#Does_project_cost_exceed_Minor_Program_limits)



## [Export Internal Check Summary](#Export_internal_check_summary)
* internal check summary (csv)


## [Final Clean Up](#FinalCleanUp)


In [2]:

from datetime import datetime
import os.path

# import requests
import pandas as pd

import numpy as np
import re

import shutil

In [3]:
import time
start_time = time.time()

In [4]:
#show dataframe without skip column
pd.options.display.max_columns = 100

In [5]:
# from config_datasource import *
import projectbookcheck_utilityfunction as uf

You are using the Extract API 2.0, please save the output as .hyper format


<a id='GlobalConstants'></a>
## Global Constants

In [6]:
# # use 'csv' to read data from data lake, use 'live' to read data directly from AmTool Server
# DATA_SOURCE_TYPE = 'csv'

# # DATALAKE_FOLDER = r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Tableau Dashboards\DataLake'

# #input data
# DATALAKE_FOLDER = r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Tableau Dashboards\DataLake'
# PROJECTBOOKCHECK_INPUT_FOLDER = r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\excel'

# #output data
# DATALAKE_HTTPSEVER_FOLDER = 'C:\inetpub\wwwroot\DataLake\ProjectBookCheck'
# PROJECTBOOKCHECK_OUTPUT_FOLDER = r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\output_internal'

# #log data
# log_folder = r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\output_internal\log'

# TARGET_FY = 2021


# # CURRENT_FY

# TARGETDATE = datetime.today().strftime("%m-%d-%Y")

In [7]:
from constants import *

In [8]:
filename = 'TenYrShopp_RawData_'
path_to_file = r'{}\{}.csv'.format(DATALAKE_HTTPSERVER_FOLDER, filename)
t = os.path.getmtime(path_to_file)

# File_TimeStamp = datetime.fromtimestamp(t).strftime("%m-%d-%Y_%H-%M")
Data_TimeStamp = datetime.fromtimestamp(t).strftime("%m-%d-%Y %H:%M:%S")

TARGETDATE = datetime.fromtimestamp(t).strftime("%m-%d-%Y")

In [9]:
CURRENT_FY = uf.fiscalyear(datetime.today())

In [10]:
# filename = 'Programming_Summary_'
# df_Programming_Summary = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, TARGETDATE))

<a id='Read_Data'></a>

# Read Data


In [11]:
File_TimeStamp = ''

if DATA_SOURCE_TYPE == 'csv':
    filename = 'Minor_Project_Details_Raw_Data_'
    df_Minor_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp))

    filename = 'Minor_Performance_Raw_Data_'
    df_Minor_perf_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp))

    filename = 'Programming_Summary_'
    df_Programming_Summary = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp))

    filename = 'Minor_Project_Postmile_Check_'
    df_Minor_pm_check = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp), header = 0)

    
else:
    print('skip getting csv data.')

In [12]:
filename = 'Minor_Activity_Crosswork.xlsx'

df_Activity_CrossWalk = pd.read_excel(r'{}\{}'.format(PROJECTBOOKCHECK_INPUT_FOLDER, filename))

# df_Activity_CrossWalk['Activity Category'].unique()

temp = df_Activity_CrossWalk.groupby('Main Activity Category')['ActID'].agg(list).reset_index()

dict_Activity_CW = dict(zip(temp['Main Activity Category'], temp['ActID']))

In [13]:
#question answered: 2021 and 2022 approved list project id duplication will be resolved with later excel files

# Data quality check and cleaning

<a id='Minor_Raw_Data'></a>
## Minor Raw Data

In [14]:
dict_rename = {'Project ID':'EFIS',
               'ID': 'AMT_ID', 
              'FY.1': 'FY_ALN',
               'Prog Appr Date': 'Prog Appr Date_ALN',
               'FY': 'FY_WP',
               'Prog Approval Date': 'Prog Appr Date_WP',
              }
df_Minor_raw_data = df_Minor_raw_data.rename(dict_rename, axis = 1)

In [15]:
df_Minor_raw_data.shape

(1263, 78)

In [16]:
# for programmed FY year of 9999, skip all the checks

# No need to check, since the raw data is filtered before download

In [17]:

df_Minor_raw_data['District'] = df_Minor_raw_data['District'].apply(uf.remove_punction)
df_Minor_raw_data['District'] = df_Minor_raw_data['District'].astype(int)

<a id='Minor_Perf_RawData'></a>
## Minor_Perf_RawData

In [18]:
#rename columns
dict_rename_perf_rawdata = {
                           'ID': 'AMT_ID',
#                             'ProjectedRTL FY': 'Projected RTL FY',

              }

df_Minor_perf_raw_data = df_Minor_perf_raw_data.rename(dict_rename_perf_rawdata, axis = 1)

In [19]:
cols_strip = ['EA','EFIS']
for c in cols_strip :
    df_Minor_perf_raw_data[c] = df_Minor_perf_raw_data[c].str.strip("'")

In [20]:
#data clean 
#data type regulation

df_Minor_perf_raw_data['Quantity'] = df_Minor_perf_raw_data['Quantity'].fillna(0)
df_Minor_perf_raw_data['Assets in Good Cond'] = df_Minor_perf_raw_data['Assets in Good Cond'].fillna(0)
df_Minor_perf_raw_data['Assets in Fair Cond'] = df_Minor_perf_raw_data['Assets in Fair Cond'].fillna(0)
df_Minor_perf_raw_data['Assets in Poor Cond'] = df_Minor_perf_raw_data['Assets in Poor Cond'].fillna(0)
df_Minor_perf_raw_data['New Assets Added'] = df_Minor_perf_raw_data['New Assets Added'].fillna(0)

# df_Minor_perf_raw_data['EFIS'] = df_Minor_perf_raw_data['EFIS'].apply(regulate_EFIS)
df_Minor_perf_raw_data['EFIS'] = pd.to_numeric(df_Minor_perf_raw_data['EFIS'], errors='coerce')


In [21]:
#data trimming
#row
df_Minor_perf_raw_data= df_Minor_perf_raw_data[df_Minor_perf_raw_data['District'] != 56]
#column
df_Minor_perf_raw_data.drop(['PID Cycle', 'TYP','ProjectedSHOPP Cycle','RequestedRTL FY','DistrictPriority'],
  axis='columns', inplace=True, errors='ignore')

In [22]:
df_Minor_perf_raw_data.name = 'df_Minor_perf_raw_data'

<a id='Counties'></a>
## Counties


In [23]:
filename = 'Counties.xlsx'

df_counties = pd.read_excel(r'{}\{}'.format(PROJECTBOOKCHECK_INPUT_FOLDER, filename))

In [24]:
df_counties['Co. Name Abbr.'] = df_counties['Co. Name Abbr.'].str.upper()

In [25]:
# df_counties.shape

In [26]:
df_counties.name = 'df_counties'

In [27]:
# df_perf_raw_prog_county = df_perf_raw_prog_candidate.merge(df_counties, how = 'left', left_on = 'County', right_on = 'Co. Name Abbr.')

In [28]:
#no need for the following, already added to the df_Minor_perf_raw_data

# #rename columns
# dict_rename_4= {
#                'Performance Objective':'Performance Objective Original', 
#               }

# df_perf_raw_prog_county = df_perf_raw_prog_county.rename(dict_rename_4, axis = 1)

<a id='Postmile_Check'></a>
## Postmile Check

In [29]:
dict_PM_ck_rename = {
 'ID': 'AMT_ID',
 '№': 'No'                            }
df_Minor_pm_check.rename(dict_PM_ck_rename, axis = 1, inplace = True)

In [30]:
df_Minor_pm_check['District'] = df_Minor_pm_check['District'].str.strip("'")
df_Minor_pm_check['District'] =df_Minor_pm_check['District'].astype(int)
df_Minor_pm_check = df_Minor_pm_check[df_Minor_pm_check['District']!= 56]

In [31]:
df_Minor_pm_check.name = 'df_Minor_pm_check'
df_Minor_pm_check.shape

(1623, 29)

<a id='ProgrammingSummary'></a>
## Programming Summary

In [32]:

dict_renamee = {'ID': 'AMT_ID',
                               }
df_Programming_Summary.rename(dict_renamee, axis = 1, inplace = True)

In [33]:
cols_strip = ['EA','EFIS']
for c in cols_strip :
    df_Programming_Summary[c] = df_Programming_Summary[c].str.strip("'")
    
df_Programming_Summary['EFIS'] = df_Programming_Summary['EFIS'].apply(uf.regulate_EFIS)
df_Programming_Summary['EFIS'] = pd.to_numeric(df_Programming_Summary['EFIS'], errors='coerce')


In [34]:
# df_Programming_Summary.head()

# Approved Project List

In [35]:
#read xlsx files
df_approved_2021 = pd.read_excel(r'{}\{}'.format('H:\Jupyter\Dev\data', 'FY2021_Minor Approved list.xlsx'))
df_approved_2022 = pd.read_excel(r'{}\{}'.format('H:\Jupyter\Dev\data', 'FY2022_Minor Approved list.xlsx'))

In [36]:
df_approved_2021['In the 2021 Approved List?'] = 'Yes'
df_approved_2022['In the 2022 Approved List?'] = 'Yes'

In [37]:
#question answered: should 2021 and 2022 approved list be treated seperately?


In [38]:
dict_rename = {
    'Project ID':'EFIS',
    'Total Project Cost ($K)': 'Construction Capital Cost ($K)'
}
df_approved_2021 = df_approved_2021.rename(dict_rename, axis = 1)


In [39]:
dict_rename = {
    'Project ID':'EFIS',
    'Contruction': 'Construction Capital Cost ($K)'
              }
df_approved_2022 = df_approved_2022.rename(dict_rename, axis = 1)

In [40]:
df_approved_2021['EFIS'] = df_approved_2021['EFIS'].apply(uf.regulate_EFIS)
df_approved_2022['EFIS'] = df_approved_2022['EFIS'].apply(uf.regulate_EFIS)

In [41]:
df_approved_2021['EFIS'] = pd.to_numeric(df_approved_2021['EFIS'], errors='coerce')
df_approved_2022['EFIS'] = pd.to_numeric(df_approved_2022['EFIS'], errors='coerce')

In [42]:
df_approved_2021['Approve Year'] = 21
df_approved_2022['Approve Year'] = 22

In [43]:
target_cols = ['EFIS','EA','Performance Value','Performance Measure','Approve Year','Program Code','Construction Capital Cost ($K)']

df_approved = df_approved_2021[target_cols].append(df_approved_2022[target_cols])
# only use 21 to check if project is in both 21 and 22
df_approved = df_approved.sort_values(by =['EFIS','Approve Year'], ascending = True)
df_approved = df_approved.groupby('EFIS').first().reset_index()

df_approved['In the Approved List?'] = 'Yes'

<a id='AddDataColumns'></a>
## Calculate and join additional fields


In [44]:
#this logic needs to consider the programming list
df_Minor_raw_data['Section'] = df_Minor_raw_data['Section In Use']

df_Minor_raw_data['Unique EA'] = df_Minor_raw_data.apply(uf.calc_unique_EA, axis = 1)

df_Minor_raw_data['FY In Use'] = df_Minor_raw_data['FY.2'].str[-2:]

In [45]:
#filter data to keep Minor program and active section only.
# df_Programming_Summary
print(df_Programming_Summary.shape)
df_Programming_Summary_filtered = pd.merge(df_Programming_Summary, df_Minor_raw_data[['AMT_ID','Section',]],
               how= 'inner', left_on = ['AMT_ID','Section',], right_on = ['AMT_ID','Section',])
print(df_Programming_Summary_filtered.shape)

(15245, 25)
(1020, 25)


In [46]:
print(df_Programming_Summary_filtered.shape)
df_Programming_Summary_filtered = pd.merge(df_Programming_Summary_filtered, df_approved,
               how= 'left', left_on = ['EFIS'], right_on = ['EFIS'],
               suffixes=['','_ApprovedList'])
print(df_Programming_Summary_filtered.shape)
df_Programming_Summary_filtered['In the Approved List?'].fillna('No', inplace=True)

(1020, 25)
(1020, 32)


In [47]:
ck_col = 'Matches Minor Approved List Performance Measure?'

def ck_performance_measure(df):
    if pd.isna(df['Performance Measure_ApprovedList']):
        return 'Not in the Approved Lists'
    else:
        if df['Performance Measure_ApprovedList'] == df['Performance Measure']:
            return 'Yes'
        else:
            return 'No'

df_Programming_Summary_filtered[ck_col]= df_Programming_Summary_filtered.apply(ck_performance_measure, axis = 1)

In [48]:

ck_col = 'Matches Minor Approved List Performance Value?'
def ck_performance_value(df):
    if pd.isna(df['Performance Value_ApprovedList']):
        return 'Not in the Approved Lists'
    else:
        if df['Performance Value_ApprovedList'] == df['Performance Value']:
            return 'Yes'
        else:
            return 'No'

df_Programming_Summary_filtered[ck_col]= df_Programming_Summary_filtered.apply(ck_performance_value, axis = 1)

In [49]:
ck_col = 'Matches Minor Approved List Performance Value and Measure?'
def ck_performance(df):
    if df['Matches Minor Approved List Performance Value?'] == 'Not in the Approved Lists':
        return 'Not in the Approved Lists'
    elif (df['Matches Minor Approved List Performance Value?'] == 'Yes') and (df['Matches Minor Approved List Performance Measure?'] == 'Yes'):
        return 'Yes'
    else:
        return 'No'
    

df_Programming_Summary_filtered[ck_col]= df_Programming_Summary_filtered.apply(ck_performance, axis = 1)

# Check Minor Data

In [50]:
df_Minor_raw_data['Program Code in Use'] = df_Minor_raw_data.apply(lambda x: x['Program Code'] if x['Section In Use'] == 'WP' else x['Program Code.1'], axis = 1)

df_Minor_raw_data['Const Capital in Use'] = df_Minor_raw_data.apply(lambda x: x['Construction Capital ($K)'] if x['Section In Use'] == 'WP' else x['Total Capital Project Cost ($K)'], axis = 1)

In [51]:
#question answered: we focus on checking the data only in the Section in Use

In [52]:
# df_Minor_raw_data_backup = df_Minor_raw_data.copy()

# df_Minor_raw_data = df_Minor_raw_data_backup

In [53]:
print(df_Minor_raw_data.shape)

df_Minor_raw_data = pd.merge(df_Minor_raw_data, df_approved[['EFIS','EA','In the Approved List?','Approve Year','Program Code','Construction Capital Cost ($K)' ]],
                            how = 'left', left_on = 'EFIS', right_on = 'EFIS', suffixes=['','_ApprovedList'])

print(df_Minor_raw_data.shape)

df_Minor_raw_data['In the Approved List?'].fillna('No', inplace= True)


(1263, 83)
(1263, 88)


In [54]:
# df_Minor_raw_data.columns

In [55]:
def ck_match_2022_approved_list(df):
    if df['In the Approved List?'] == 'Yes' and df['Approve Year'] == 22:
        if df['FY In Use'] == 22:
            return 'OK'
        else:
            return r'The FY {} does not match Approved year {}'.format(df['FY In Use'], df['Approve Year'])
    else:
        return 'Not in the 2022 Approved list'

ck_col = 'FY Matches 2022 List?'
df_Minor_raw_data[ck_col] = df_Minor_raw_data.apply(ck_match_2022_approved_list, axis = 1)

### is EFIS duplicate within Minor raw data?

In [56]:
# temp = df_Minor_raw_data.groupby(['EFIS'])['AMT_ID'].nunique().reset_index(name = 'EFIS_Counts')
# duplicated_EFIS= temp[temp['EFIS_Counts']> 1]

# df_Minor_raw_data.drop(columns=['EFIS_Counts'],inplace=True , errors='ignore')
# print(df_Minor_raw_data.shape)
# df_Minor_raw_data = pd.merge(df_Minor_raw_data, duplicated_EFIS, 
#                              how = 'left', left_on = ['EFIS'], right_on=['EFIS'])
# print(df_Minor_raw_data.shape)

# def ck_EFIS_Uniqueness(df):
#     if pd.isna(df['EFIS_Counts']):
#         return 'OK'
#     elif df['EFIS'] == 0: 
#         return 'Missing/Invalid EFIS'
#     else:
#         return 'Duplicate EFIS'
    
# df_Minor_raw_data['EFIS Uniqueness Check'] = df_Minor_raw_data.apply(ck_EFIS_Uniqueness, axis = 1)

In [57]:
temp = df_Minor_raw_data.groupby(['EFIS'])['AMT_ID'].agg([pd.Series.nunique, list]).reset_index()
temp['AMT_IDs'] = temp['list'].apply(lambda l: ','.join(l))
duplicated_EFIS= temp[temp['nunique']> 1]

df_Minor_raw_data.drop(columns=['nunique','AMT_IDs'],inplace=True , errors='ignore')
print(df_Minor_raw_data.shape)
df_Minor_raw_data = pd.merge(df_Minor_raw_data, duplicated_EFIS, 
                             how = 'left', left_on = ['EFIS'], right_on=['EFIS'])
print(df_Minor_raw_data.shape)

def ck_EFIS_Uniqueness(df):
    if pd.isna(df['nunique']):
        return 'OK'
    elif df['EFIS'] == 0: 
        return 'Missing/Invalid EFIS'
    else:
        return 'Duplicate EFIS {} is found in the following projects: {}'.format(df['EFIS'], df['AMT_IDs'])
    
df_Minor_raw_data['EFIS Uniqueness Check'] = df_Minor_raw_data.apply(ck_EFIS_Uniqueness, axis = 1)

(1263, 89)
(1263, 92)


In [58]:
# df_Minor_raw_data['EFIS'].value_counts()

In [59]:
# df_Minor_raw_data['EFIS Uniqueness Check'].value_counts()

In [60]:
# df_Minor_raw_data['EA']

In [61]:
# #flag if EFIS is invalid
# def ck_invalid_EFIS(df):
#     if len(str(EFIS)) < 5: 
#         return 'Invalid EFIS'
#     else:
#         return 'OK'
# df_Minor_raw_data['EFIS is valid?'] = df_Minor_raw_data['EFIS'].apply(ck_invalid_EFIS)

In [62]:
# df_Minor_raw_data.columns

### flag if District + EA duplicate within Minor raw data

In [63]:
temp = df_Minor_raw_data.groupby(['Unique EA'])['AMT_ID'].nunique().reset_index(name = 'UnqiueEA_Counts')
duplicated_EA= temp[temp['UnqiueEA_Counts']> 1]

df_Minor_raw_data.drop(columns=['UnqiueEA_Counts'],inplace=True , errors='ignore')

df_Minor_raw_data = pd.merge(df_Minor_raw_data, duplicated_EA[['Unique EA','UnqiueEA_Counts']].drop_duplicates(), 
                             how = 'left', left_on = ['Unique EA'], right_on=['Unique EA'])

def ck_EA_Uniqueness(df):
    if pd.isna(df['UnqiueEA_Counts']):
        return 'OK'
    else:
        return 'Duplicate District+EA is found.'
    
df_Minor_raw_data['EA Uniqueness Check'] = df_Minor_raw_data.apply(ck_EA_Uniqueness, axis = 1)

In [64]:
ck_col = 'Does Project have a Repeated EA or Project ID repeated in Minor Profile?'

def ck_ID_Uniqueness(df):
    if df['EA Uniqueness Check'] == 'OK' and df['EA Uniqueness Check'] == 'OK':
        return 'OK'
    else:
        return 'Duplicate District+EA and/or Project ID(EFIS) is found.'
    
df_Minor_raw_data[ck_col] = df_Minor_raw_data.apply(ck_ID_Uniqueness, axis=1 )

In [65]:
# Does FY Need Updates?

def ck_FY_consistancy(df):
    if df['In the Approved List?'] == 'Yes':
        if df['FY In Use'] == df['Approve Year']:
            return 'OK'
        else:
            return 'Please update FY. It is in the {} Approved List'.format(df['Approve Year'])
    else:
        return 'OK'
    
df_Minor_raw_data['Does FY Need Updates?'] = df_Minor_raw_data.apply(ck_FY_consistancy, axis = 1)


In [66]:
# Does EA Need Updates?

def ck_EA_consistancy(df):
    
    if df['EA'] == df['EA_ApprovedList']:
        return 'OK'
    else:
        return 'Update EA. It does not match EA in Approved List of year {}'.format(df['Approve Year'])
    
df_Minor_raw_data['Does EA Need Updates?'] = df_Minor_raw_data.apply(ck_EA_consistancy, axis = 1)

In [67]:
#Does Program Code Need Updates?

ck_col = 'Does Program Code Need Updates?'

def ck_program_code_update(df):
    if pd.isna(df['Program Code_ApprovedList']) or (df['Program Code in Use'] == df['Program Code_ApprovedList']):
        return 'OK'
    else:
        return 'The program code for Section In Use does not match Approved project list.'

df_Minor_raw_data[ck_col] = df_Minor_raw_data.apply(ck_program_code_update, axis = 1)

In [68]:
# ck_col = 'Does Program Code Need Updates?'
# df_Minor_raw_data[ck_col].value_counts()

In [69]:
# df_Activity_CrossWalk['Activity Category'].unique()

In [70]:
# for a in df_Activity_CrossWalk['Activity Category'].unique():
#     if a not in df_Minor_perf_raw_data['Main Activity Category'].unique():
#         print (a)

In [71]:
# for a in df_Minor_perf_raw_data['Main Activity Category'].unique():
#     if a not in df_Activity_CrossWalk['Activity Category'].unique():
#         print (a)

In [72]:
df_Activity_CrossWalk['QualifiedActivity'] = 'Yes'

temp = pd.merge(df_Minor_perf_raw_data, df_Activity_CrossWalk, 
         how='left', left_on=['Main Activity Category', 'ActID'], right_on=['Main Activity Category', 'ActID'])

temp['QualifiedActivity'].fillna('No', inplace=True)

temp = temp.groupby(['AMT_ID', 'Section'])['QualifiedActivity'].agg(list).reset_index()


temp['Project Has Qualified Activity?'] = temp['QualifiedActivity'].apply(
    lambda x: 'Yes' if 'Yes' in x else 'No'
)

,AMT_ID,Section,QualifiedActivity,Project Has Qualified Activity?
0,MA000001,ALN,"[Yes, Yes]",Yes
1,MA000001,WP,"[Yes, Yes]",Yes
2,MA000002,ALN,[No],No
3,MA000002,WP,"[Yes, Yes]",Yes
4,MA000003,ALN,"[Yes, No]",Yes


In [74]:
df_Minor_raw_data.columns

Index(['Status', 'AMT_ID', 'Minor', 'EFIS', 'EA', 'District', 'County',
       'Route', 'Begin PM', 'End PM', 'Multiple Location', 'Program Code',
       'Activity Category', 'Minor Activity', 'FCO Only', 'Planned RTL Date',
       'FY_WP', 'PA&ED, PS&E, R/W Sup', 'Const Eng Sup ($K)',
       'R/W Capital ($K)', 'Construction Capital ($K)',
       'Total Project Cost ($K)', 'Prog Appr Date_WP', 'Combine',
       'Project Location/Description', 'Comments', 'Project ID.1', 'EA.1',
       'District.1', 'County.1', 'Route.1', 'Begin PM.1', 'End PM.1',
       'Program Code.1', 'Activity Category.1', 'Minor Activity.1',
       'Coop Agreement', 'Actual RTL Date', 'FY_ALN', 'Allocation Date',
       'Const Eng Sup', 'Const Cap state ($K)', 'Const Cap Fed ($K)',
       'Total Capital Project Cost ($K)', 'Prog Appr Date_ALN', 'Fiscal Year',
       'CTC Meeting', 'Resolution No', 'Rescission Date',
       'Project Location/Description.1', 'Comments.1', 'Contract Approval',
       'Federal Aid Pr

In [75]:
print(df_Minor_raw_data.shape)
df_Minor_raw_data = pd.merge(df_Minor_raw_data, temp[['AMT_ID', 'Section', 'Project Has Qualified Activity?']], 
         how='left', left_on=['AMT_ID', 'Section'], right_on=['AMT_ID', 'Section'])
print(df_Minor_raw_data.shape)

(1263, 99)
(1263, 100)


,Status,AMT_ID,Minor,EFIS,EA,District,County,Route,Begin PM,End PM,Multiple Location,Program Code,Activity Category,Minor Activity,FCO Only,Planned RTL Date,FY_WP,"PA&ED, PS&E, R/W Sup",Const Eng Sup ($K),R/W Capital ($K),Construction Capital ($K),Total Project Cost ($K),Prog Appr Date_WP,Combine,Project Location/Description,Comments,Project ID.1,EA.1,District.1,County.1,Route.1,Begin PM.1,End PM.1,Program Code.1,Activity Category.1,Minor Activity.1,Coop Agreement,Actual RTL Date,FY_ALN,Allocation Date,Const Eng Sup,Const Cap state ($K),Const Cap Fed ($K),Total Capital Project Cost ($K),Prog Appr Date_ALN,Fiscal Year,CTC Meeting,Resolution No,Rescission Date,Project Location/Description.1,Comments.1,Contract Approval,Federal Aid Project,Certified Small Business,Prime Contractor,Project Allotment ($K),Federal Total ($K),Small Business Amount ($K),Prog Appr Date.1,1st FY,1st Removal Date,2nd FY,2nd Removal Date,3rd FY,3rd Removal Date,4th FY,4th Removal Date,5th FY,5th Removal Date,6th FY,6th Removal Date,No of Workplans,Comments.2,Last Saved,Saved By,Section In Use,FY.2,Project Cost In Use ($K),Section,Unique EA,FY In Use,Program Code in Use,Const Capital in Use,EA_ApprovedList,In the Approved List?,Approve Year,Program Code_ApprovedList,Construction Capital Cost ($K),FY Matches 2022 List?,nunique,list,AMT_IDs,EFIS Uniqueness Check,UnqiueEA_Counts,EA Uniqueness Check,Does Project have a Repeated EA or Project ID repeated in Minor Profile?,Does FY Need Updates?,Does EA Need Updates?,Does Program Code Need Updates?,Project Has Qualified Activity?
0,Active,MA000001,A,113000078.0,0C930,1,HUM,'101,79.9,84.9,No,201.151,Drainage,NaN,NaN,01/01/00,2019/20,0.0,0.0,25.0,909.0,934.0,06/30/20,NaN,"In Humboldt County, in and near Eureka and Arc...",NaN,113000078.0,0C930,'01,HUM,101.0,79.9,84.9,201.151,Drainage,NaN,NaN,06/30/20,2019/20,06/30/20,0.0,652.0,0.0,652.0,06/30/20,2019/20,Jun 2020,NaN,01/01/00,"In Humboldt County, in and near Eureka and Arc...",NaN,06/30/20,NaN,NaN,NaN,496.5,0.0,0.0,01/01/00,NaN,01/01/00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,04/22/20,Mara Chaudhari,ALN,2019/20,652.0,ALN,1_0C930,20,201.151,652.0,NaN,No,NaN,NaN,NaN,Not in the 2022 Approved list,NaN,NaN,NaN,OK,NaN,OK,OK,OK,Update EA. It does not match EA in Approved Li...,OK,Yes
1,Active,MA000002,A,115000064.0,0F080,1,HUM,'096,22.9,22.9,No,201.015,Safety - Collision Reduction,NaN,NaN,01/01/00,2019/20,0.0,0.0,5.0,1103.0,1108.0,06/30/20,NaN,"In Humboldt County, in Weitchpec at Pearson La...",NaN,115000064.0,0F080,'01,HUM,96.0,22.9,22.9,201.015,Safety - Collision Reduction,NaN,NaN,06/30/20,2019/20,06/30/20,0.0,1186.0,0.0,1186.0,06/30/20,2019/20,Jun 2020,NaN,01/01/00,"In Humboldt County, in Weitchpec at Pearson La...",NaN,06/30/20,NaN,NaN,NaN,1340.0,0.0,0.0,01/01/00,NaN,01/01/00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,09/10/19,Graciela Vasquez,ALN,2019/20,1186.0,ALN,1_0F080,20,201.015,1186.0,NaN,No,NaN,NaN,NaN,Not in the 2022 Approved list,NaN,NaN,NaN,OK,NaN,OK,OK,OK,Update EA. It does not match EA in Approved Li...,OK,No
2,Active,MA000003,A,118000146.0,0J010,1,MEN,'001,59.3,59.3,No,201.121,Pavement,NaN,NaN,01/01/00,2019/20,0.0,0.0,0.0,1050.0,1050.0,06/30/20,NaN,"In Mendocino County, near Fort Bragg at Simpso...",NaN,118000146.0,0J010,'01,MEN,1.0,59.3,59.3,201.121,Pavement,NaN,NaN,06/30/20,2019/20,06/30/20,0.0,1136.0,0.0,1136.0,06/30/20,2019/20,Jun 2020,NaN,01/01/00,"In Mendocino County, near Fort Bragg at Simpso...",NaN,06/30/20,NaN,NaN,NaN,1036.0,0.0,0.0,01/01/00,NaN,01/01/00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,09/10/19,Graciela Vasquez,ALN,2019/20,1136.0,ALN,1_0J010,20,201.121,1136.0,NaN,No,NaN,NaN,NaN,Not in the 2022 Approved list,NaN,NaN,NaN,OK,NaN,OK,OK,OK,Update EA. It does not match EA in Approved Li...,OK,Yes
3,Active,MA000004,A,215000006.0,0H390,2,TEH,'036,87,87.8,No,201.315,Mobility - TMS,NaN,NaN,03/13/19,2019/20,320.0,0.0,98.0,719.0,1137.0,06/30/20,NaN,Morgan Summit CCTV & RWIS In Tehama County nea...,"PM Javed Iqbal, 530-945-1932 (

In [78]:
ck_name = 'Project has at least one qualified activity?'

def ck_qualified_activity(df):
    if pd.isna(df['Project Has Qualified Activity?']):
        return 'The performance data is missing for this project.'
    elif df['Project Has Qualified Activity?'] == 'No':
        try: 
            act_list = ','.join(dict_Activity_CW[df['Activity Category']])
        except:
            act_list = 'please find out from HQ AM for the latest list.'
        return 'Please update the performance to include at least one qualified activity. The qualified activity for {} are {}'.format(df['Activity Category'], act_list)
    else:
        return 'OK'

df_Minor_raw_data[ck_name] = df_Minor_raw_data.apply(ck_qualified_activity, axis=1)

In [79]:
# df_Minor_raw_data[df_Minor_raw_data['Does Program Code Need Updates?'] != 'OK'][['AMT_ID','Program Code in Use','Program Code_ApprovedList']]

In [80]:
# #TODO:

# cols = ['EA','EFIS','SYSNO', 'INETNO',
#        'OUTETNO',]

# for c in cols:
#     df_Minor_drain_raw_data[c] = df_Minor_drain_raw_data[c].apply(uf.remove_punction)

In [81]:
# #TODO
# #check for each Main Activity Category, at least on ActID within the cross walk list provided by mara
# # 
# temp = df_Minor_drain_raw_data.groupby(['EFIS','Section'])['ActID'].agg(lambda l: ','.join(l)).reset_index(name = 'ActIDs')
# temp['Drainage Activity Check'] = temp['ActIDs'].apply(lambda x: 'OK' if 'C' in x else 'This project needs to have at least one drainage activity starts with "C".')

# df_Minor_brg_raw_data.head()

# df_Minor_brg_raw_data.columns

# #question to be answered: there is not ActID in the bridge worksheet raw data

# cols = ['District.1','Route']

# for c in cols:
#     df_Minor_pav_raw_data[c] = df_Minor_pav_raw_data[c].apply(uf.remove_punction)

# dict_rename = {
#     'ID':'AMT_ID'
# }

# df_Minor_pav_raw_data.rename(dict_rename, axis = 1, inplace=True)

# df_Minor_pav_raw_data.head()

# #TODO
# #check for each Main Activity Category, at least on ActID within the cross walk list provided by mara
# # 

# # pavement: needs to have at least one B activity id
# temp = df_Minor_pav_raw_data.groupby(['AMT_ID','Section'])['ActID'].agg(lambda l: ','.join(l)).reset_index(name = 'ActIDs')
# temp['Pavement Activity Check'] = temp['ActIDs'].apply(lambda x: 'OK' if 'B' in x else 'This project needs to have at least one pavement activity starts with "B".')

# temp['Pavement Activity Check'].unique()

# # bridge,needs to have at least one A activity id
# # temp = df_Minor_drain_raw_data.groupby(['EFIS','Section'])['ActID'].agg(lambda l: ','.join(l)).reset_index(name = 'ActIDs')
# # temp['Drainage Activity Check'] = temp['ActIDs'].apply(lambda x: 'OK' if 'C' in x else 'This project needs to have at least one drainage activity starts with "C".')

In [82]:
#question to be answered: 
# safety: needs to have at least one within the list []

In [83]:
# def ck_shape(*args, **kwargs):
#     def wrapper_func(original_func):
#         print (kwargs['df'].shape)
#         results = original_func(*args, **kwargs)
#         print (kwargs['df']..shape)
#         return results
#     return wrapper_func

# @ck_shape(df = df_Minor_raw_data)
# def add(x,y):
#     return (x+y) 



In [84]:
#Does Construction Capital Cost ($K) Need Updates?

ck_col = 'Does Construction Capital Cost ($K) Need Updates?'

def ck_construction_capital_cost(df):
       
    if abs(df['Construction Capital Cost ($K)'] - df['Const Capital in Use']) < 0.01:
        return 'OK'
    else:
        # question to be answered: for construct cost ck, for 22, only check WP band if Section in WP
        if df['Section'] == 'ALN' or (df['Section'] == 'WP' and df['Approve Year'] == 22): 
            return 'Update Capital Cost. It does not match Approved List'
        else:
            return 'OK'
        
df_Minor_raw_data[ck_col] = df_Minor_raw_data.apply(ck_construction_capital_cost, axis = 1)

### flag if no performance
 performance value can be zero

In [85]:
#TODO
# FY in Use needs to be the same as the pavement, TMS worksheet plan year

# flag the invalid locations


In [86]:
ck_col = 'Was Performance Tab Completed in Section in Use?'

temp = df_Minor_perf_raw_data.groupby(['AMT_ID','Section']).first().reset_index()
temp['Has performance raw data?'] = 'Yes'

df_Minor_raw_data.drop(columns=['Has performance raw data?',],inplace=True , errors='ignore')
print(df_Minor_raw_data.shape)

df_Minor_raw_data = pd.merge(df_Minor_raw_data, temp[['AMT_ID','Section','Has performance raw data?']].drop_duplicates(), 
                             how = 'left', left_on = ['AMT_ID','Section'], right_on=['AMT_ID','Section'])

df_Minor_raw_data['Has performance raw data?'].fillna('No', inplace=True)

print(df_Minor_raw_data.shape)


def ck_performance_availability(df):
    if df['Has performance raw data?'] == 'No':
        return '"Please complete Performance Tab in Section {}'.format(df['Section'])
    else:
        return 'OK'

df_Minor_raw_data[ck_col] = df_Minor_raw_data.apply(ck_performance_availability, axis = 1)

(1263, 102)
(1263, 103)


In [87]:
# Does Performance in Section in Use Match Approved List?
# check shape
print(df_Minor_raw_data.shape)
#remove column

col_name = 'Matches Minor Approved List Performance Value and Measure?'
df_Minor_raw_data.drop(columns=[col_name],inplace=True , errors='ignore')

#join
df_Minor_raw_data = pd.merge(
    df_Minor_raw_data, 
    df_Programming_Summary_filtered[['AMT_ID', 'Section',col_name]],
    how='left', left_on=['AMT_ID', 'Section'], right_on=['AMT_ID', 'Section']
)
#fill na
#question to be answered: for projects not in the programming summary list, we assigned the performance value and measure check to No

df_Minor_raw_data[col_name].fillna('No', inplace=True)

print(df_Minor_raw_data.shape)

(1263, 104)
(1263, 105)


In [88]:
ck_col = 'Was project with FY Before 2021/22 Closed-Out?'

def ck_project_closeout_status(df):
    if pd.isna(df['FY In Use']):
        return 'Please Identify FY'
    elif int(df['FY In Use']) < 22 and df['Section'] == 'ALN':
        return 'OK'
    else:
        return 'Please work with HQ Minor Program to Close-out Project'
    
df_Minor_raw_data[ck_col] = df_Minor_raw_data.apply(ck_project_closeout_status, axis=1)

In [89]:
ck_col = 'Data Needs Review?'

input_cols = ['Does Project have a Repeated EA or Project ID repeated in Minor Profile?',
       'Does FY Need Updates?', 'Does EA Need Updates?',
       'Does Program Code Need Updates?',
       'Does Construction Capital Cost ($K) Need Updates?',
       'Has performance raw data?',
       'Was Performance Tab Completed in Section in Use?',
       'Matches Minor Approved List Performance Value and Measure?',
       'Was project with FY Before 2021/22 Closed-Out?']

def ck_review_needs(df, input_cols):
    for col in input_cols:
        if df[col] != 'OK:':
            return 'District needs to review project data (Profile and/or RTL)'
    return 'OK'
    
df_Minor_raw_data[ck_col] = df_Minor_raw_data.apply(ck_review_needs, args = [input_cols], axis=1)


In [90]:

ck_col = 'Data needs review other that Close-out?'

input_cols = ['Does Project have a Repeated EA or Project ID repeated in Minor Profile?',
       'Does FY Need Updates?', 'Does EA Need Updates?',
       'Does Program Code Need Updates?',
       'Does Construction Capital Cost ($K) Need Updates?',
       'Has performance raw data?',
       'Was Performance Tab Completed in Section in Use?',
       'Matches Minor Approved List Performance Value and Measure?',
#          'Was project with FY Before 2021/22 Closed-Out?'     
       ]

def ck_review_needs_2(df, input_cols):
    for col in input_cols:
        if df[col] != 'OK:':
            return 'District needs to review project data (Profile and/or RTL)'
    return 'OK'
    
df_Minor_raw_data[ck_col] = df_Minor_raw_data.apply(ck_review_needs_2, args = [input_cols], axis=1)

In [91]:
#question to be answered, can we convert the following checks into "OK" or others, to be used in filter out flagged items in the punchlist.


In [92]:
ck_col = 'Was information Entered in the Allocation Band?'

def ck_ALN_band_info_completeness(df):
    if pd.isna(df['FY_ALN']) or df['Has performance raw data?'] == 'No' or pd.isna(df['Total Capital Project Cost ($K)']):
        return 'No'
    else: 
        return 'Yes'
    
df_Minor_raw_data[ck_col] = df_Minor_raw_data.apply(ck_ALN_band_info_completeness, axis=1)

In [93]:
ck_col = 'Is Project ready to enter data in the Allocation Band?'

def ck_readiness_to_enter_ALN_band(df):
    if pd.notna(df['Prog Appr Date_WP']):  #has approval date in WP band
        if df['Section'] == 'ALN':
            return 'Project was closed-out'
        elif df['Was information Entered in the Allocation Band?'] == 'Yes':
            return 'Allocation Band needs review by HQ Minor Program. If all data Accurate HQ Minor will enter the approval date'
        else:
            return 'Project ready to enter data in the Allocation Band (Cost, Schedule, RTL, And/Or Performance Tab)'
    else: 
        return 'Workplan Band needs review by HQ Minor Program. If all data Accurate HQ Minor will enter the approval date'
    
df_Minor_raw_data[ck_col] = df_Minor_raw_data.apply(ck_readiness_to_enter_ALN_band, axis=1)

In [94]:

ck_col = 'Is Project Project Ready for Review and Approval Date?'

def ck_readiness_for_review(df):
    if df['Data needs review other that Close-out?'] != 'OK':
        return 'No'
    elif df['FY In Use'] > 22: 
        return 'No'
    elif pd.isna(df['Prog Appr Date_WP']):
        return 'HQ Needs to review Workplan band and enter Approval Date if data is accurate'
    
    elif df['Was information Entered in the Allocation Band?'] == 'Yes':
        if pd.notna(df['Prog Appr Date_ALN']):
            return 'No, Project Already Closed-out'
        else:
            return 'HQ Needs to review Allocation band and enter Approval Date if data is accurate'
    else:
        return 'No'
df_Minor_raw_data[ck_col] = df_Minor_raw_data.apply(ck_readiness_for_review, axis=1)

In [95]:
ck_col = 'Does Worplan Band needs Approval Removal?'

def ck_WP_data_error(df):
    if pd.isna(df['Prog Appr Date_WP']):  #has no approval data in WP band
        return 'No'
    elif (pd.isna(df['FY In Use'])
        or int(df['FY In Use']) > 22 
        or (df['FY In Use'] in ['21', '22'] and df['In the Approved List?'] == 'No')
         ): 
        return 'HQ Minor Program needs to remove Approval date fromWorkplan Band, so District can updated the project FY. Project not in Approved lists or in the future'
    else:
        return 'No'
    
df_Minor_raw_data[ck_col] = df_Minor_raw_data.apply(ck_WP_data_error, axis=1)

In [96]:
ck_col = 'Does Allocation Band needs Approval Removal?'

def ck_ALN_data_error(df):
    if pd.isna(df['Prog Appr Date_ALN']):  #has no approval data in ALN band
        return 'No'
    elif df['Data needs review other that Close-out?'] == 'OK': 
        return 'No'
    else:
        return 'HQ Minor Program needs to remove Approval date from Allocation Band, so District can updated the project data'

df_Minor_raw_data[ck_col] = df_Minor_raw_data.apply(ck_ALN_data_error, axis=1)

In [97]:
ck_col = 'HQ Minor Program Needs Review?'

def ck_review_needs_HQ_Minor(df):
    if (df['Is Project Project Ready for Review and Approval Date?'] == "No"
        and df['Does Worplan Band needs Approval Removal?'] == 'No'
        and df['Does Allocation Band needs Approval Removal?'] == 'No'
       ):
        return 'No'
    else: 
        return "HQ Minor Needs Review"
    
df_Minor_raw_data[ck_col] = df_Minor_raw_data.apply(ck_review_needs_HQ_Minor, axis=1)

In [98]:
#question to be answered: 
# for every project in Minor raw data, the section-in-use performance needs to be filled. 
# the performance measure unit and value should match with approved project list performance meansure, if available in approved project list. 
# if the raw data has all the information needed, including FY and performance data, and it is not only the approved project list, Minor project HQ needs to review and approve the project. 
#if the project is on the approved list, the HQ needs reach out the district to get the project close out. 


In [99]:
#question to be answered: 
# do we need the following checks

### flag if total project cost is zero

In [100]:

def ck_total_project_cost(df):
    if pd.isna(df['Total Project Cost ($K)']) or df['Total Project Cost ($K)'] == 0:
        return 'Total project cost can not Empty or zero.'
    else:
        return 'OK'
    
df_Minor_raw_data['Total Project Cost Check'] = df_Minor_raw_data.apply(ck_total_project_cost, axis = 1)

### flag if project description is blank


In [101]:
def ck_project_description(df):
    if pd.isna(df['Project Location/Description']) or df['Project Location/Description'] == '':
        return 'Project Location/Description can not empty.'
    else:
        return 'OK'
    
df_Minor_raw_data['Project Location/Description Check'] = df_Minor_raw_data.apply(ck_project_description, axis = 1)

### check pm validation

In [102]:
df_Minor_pm_invalid = df_Minor_pm_check[df_Minor_pm_check['Valid PM'] != 'Yes']

AMT_IDs_withInvalidPM = df_Minor_pm_invalid['AMT_ID'].unique()

In [103]:

def ck_invalid_pm(df):
    if df['AMT_ID'] in AMT_IDs_withInvalidPM:
        return 'The PM is invalid.'
    else:
        return 'OK'
    
df_Minor_raw_data['PM Validity Check'] = df_Minor_raw_data.apply(ck_invalid_pm, axis = 1)


time elapsed : 5.867709159851074 seconds


<a id='Export_Data'></a>
# Export Data

In [105]:
# DATA_HHMM = datetime.now().strftime("%H%M")

file_export_log = open(LOG_FILE, "a")  # append mode
file_export_log.write("#####Minor Data Check:{} \n".format(Data_TimeStamp))
file_export_log.close()

In [106]:
df_Minor_raw_data['Data_TimeStamp'] = Data_TimeStamp


## export check flags

In [107]:
ck_cols = [
    'EFIS Uniqueness Check',
    'EA Uniqueness Check',
    'Does Project have a Repeated EA or Project ID repeated in Minor Profile?',
    'Does FY Need Updates?', 
    'Does EA Need Updates?',
    'Does Program Code Need Updates?',
    'Does Construction Capital Cost ($K) Need Updates?',
    'Has performance raw data?',
    'Was Performance Tab Completed in Section in Use?',
    'Matches Minor Approved List Performance Value and Measure?',
    'Was project with FY Before 2021/22 Closed-Out?', 
 
    #additional checks
    'PM Validity Check',
    'Project has at least one qualified activity?',
]


In [109]:
#export all projects with all checks in matrix
out_cols = [
    
    #project information
    'AMT_ID', 'Minor', 'EFIS', 'EA', 'District', 
    'Data_TimeStamp',
    'Data Needs Review?',
    'Data needs review other that Close-out?',
                    ]

out_cols.extend(ck_cols)


filename = 'Minor_Datachecks_Matrix'
df_out = df_Minor_raw_data[out_cols]

uf.export_csv(df_out, filename, PROJECTBOOKCHECK_HTTPSERVER_FOLDER, LOG_FILE)



In [110]:
#table 1

df_melted = pd.melt(df_Minor_raw_data, 
                    id_vars=['AMT_ID'], 
                    value_vars=ck_cols, var_name = 'Check Description')

df_melted.columns = ['AMT_ID','Check Description','Check Summary']
df_melted_filtered = df_melted[df_melted['Check Summary']!= 'OK']


out_cols = [
    
    #project information
    'AMT_ID', 'Minor', 'EFIS', 'EA', 'District', 'Data_TimeStamp'
                    ]

df_out = pd.merge(df_melted_filtered, df_Minor_raw_data[out_cols],
                  how = 'left', left_on = 'AMT_ID', right_on = 'AMT_ID')


filename = 'Minor_Datachecks_Punchlist'
uf.export_csv(df_out, filename, PROJECTBOOKCHECK_HTTPSERVER_FOLDER, LOG_FILE)

hyper_name = '{}.hyper'.format(filename)
uf.export_hyper(df_out, hyper_name, LOG_FILE)


processing table: 0it [00:00, ?it/s]

Table 'Extract' does not exist in extract Minor_Datachecks_Punchlist.hyper.hyper, creating.


processing table: 6078it [00:00, 24605.77it/s]


Signing into AssetManagement at https://tableau.dot.ca.gov
Publishing Minor_Datachecks_Punchlist.hyper.hyper to Sandbox_ProjectBookCheck_Automation...


## Export action items for Minor District Engineer

In [114]:
filename = 'Minor_District_ActionItem'

out_cols = [
    #project information
    'AMT_ID', 'Minor', 'EFIS', 'EA', 'District', 'Data_TimeStamp',
    'Is Project ready to enter data in the Allocation Band?'
                    ]
df_out = df_Minor_raw_data[out_cols]
uf.export_csv(df_out, filename, PROJECTBOOKCHECK_HTTPSERVER_FOLDER, LOG_FILE)

hyper_name = '{}.hyper'.format(filename)
uf.export_hyper(df_out, hyper_name, LOG_FILE)

processing table: 1263it [00:00, 28695.27it/s]

Table 'Extract' does not exist in extract Minor_District_ActionItem.hyper.hyper, creating.


Signing into AssetManagement at https://tableau.dot.ca.gov
Publishing Minor_District_ActionItem.hyper.hyper to Sandbox_ProjectBookCheck_Automation...


## Export action items for Minor HQ Engineer

In [115]:
filename = 'Minor_HQ_ActionItem'

out_cols = [
    #project information
    'AMT_ID', 'Minor', 'EFIS', 'EA', 'District', 'Data_TimeStamp',
    
    'Is Project Project Ready for Review and Approval Date?',
    'Does Worplan Band needs Approval Removal?',
    'Does Allocation Band needs Approval Removal?',
    'HQ Minor Program Needs Review?',
                    ]
df_out = df_Minor_raw_data[out_cols]

uf.export_csv(df_out, filename, PROJECTBOOKCHECK_HTTPSERVER_FOLDER, LOG_FILE)

hyper_name = '{}.hyper'.format(filename)
uf.export_hyper(df_out, hyper_name, LOG_FILE)

processing table: 1263it [00:00, 21047.51it/s]

Table 'Extract' does not exist in extract Minor_HQ_ActionItem.hyper.hyper, creating.


Signing into AssetManagement at https://tableau.dot.ca.gov
Publishing Minor_HQ_ActionItem.hyper.hyper to Sandbox_ProjectBookCheck_Automation...


<a id='Export_programming_summary'></a>

### Export Programming Summary

In [116]:
out_col =df_Programming_Summary_filtered.columns

filename = 'Minor_Programming_Summary'
df_out = df_Programming_Summary_filtered[out_col]
df_out['Data_TimeStamp'] = Data_TimeStamp

uf.export_csv(df_out, filename, PROJECTBOOKCHECK_HTTPSERVER_FOLDER, LOG_FILE)

hyper_name = '{}.hyper'.format(filename)
uf.export_hyper(df_out, hyper_name, LOG_FILE)

processing table: 638it [00:00, 6379.09it/s]

Table 'Extract' does not exist in extract Minor_Programming_Summary.hyper.hyper, creating.


processing table: 1020it [00:00, 6257.27it/s]


Signing into AssetManagement at https://tableau.dot.ca.gov
Publishing Minor_Programming_Summary.hyper.hyper to Sandbox_ProjectBookCheck_Automation...



<a id='FinalCleanUp'></a>
## Final Clean Up

In [117]:
#clean up tableau publishing log file

import os
import glob
# get a recursive list of file paths that matches pattern
fileList = glob.glob('./*.log')
# Iterate over the list of filepaths & remove each file.
for filePath in fileList:
    try:
        os.remove(filePath)
    except OSError:
        print("Error while deleting file")


In [118]:
end_time =  time.time()
elapsed = end_time - start_time
print('time elapsed : {} seconds'.format(elapsed))

time elapsed : 136.37857174873352 seconds
